# QRT Challenge Data 2024
## Data Specs (Copied from Official)
- `X_train` and `X_test` both have $35$ columns that represent the same explanatory variables but over different time periods. 

- `X_train` and `Y_train` share the same column `ID` - each row corresponds to a unique ID associated wwith a day and a country. 

- The target of this challenge `TARGET` in `Y_train` corresponds to the price change for daily futures contracts of 24H electricity baseload. 

- **You will notice some columns have missing values**.

Input data sets comprise 35 columns:

ID: Unique row identifier, associated with a day (DAY_ID) and a country (COUNTRY),

DAY_ID: Day identifier - dates have been anonymized, but all data corresponding to a specific day is consistent,

COUNTRY: Country identifier - DE = Germany, FR = France,
and then contains daily commodity price variations,

GAS_RET: European gas,

COAL_RET: European coal,

CARBON_RET: Carbon emissions futures,

#### Weather measures (daily, in the country x)

x_TEMP: Temperature,

x_RAIN: Rainfall,

x_WIND: Wind,


#### Energy production measures (daily, in the country x)

x_GAS: Natural gas,

x_COAL: Hard coal,

x_HYDRO: Hydro reservoir,

x_NUCLEAR: Daily nuclear production,

x_SOLAR: Photovoltaic,

x_WINDPOW: Wind power,

x_LIGNITE: Lignite,

#### Electricity use metrics (daily, in the country x)

x_CONSUMPTON: Total electricity consumption,

x_RESIDUAL_LOAD: Electricity consumption after using all renewable energies,

x_NET_IMPORT: Imported electricity from Europe,

x_NET_EXPORT: Exported electricity to Europe,

DE_FR_EXCHANGE: Total daily electricity exchange between Germany and France,

FR_DE_EXCHANGE: Total daily electricity exchange between France and Germany.

Output data sets are composed of two columns:

ID: Unique row identifier - corresponding to the input identifiers,

TARGET: Daily price variation for futures of 24H electricity baseload.


In [5]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import itertools as it
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression as mir
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder as onehot
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
import xgboost as xgb
import lightgbm as lgb
from sklearn.impute import KNNImputer

In [8]:
df = pd.read_csv('../data/X_train_NHkHMNU.csv')
y = pd.read_csv('../data/y_train_ZAN5mwg.csv')
true_test = pd.read_csv('../data/X_test_final.csv')
pd.set_option('display.max_columns', None)


In [9]:
# for fitting two part linear regression to WIND_SQCB / WINDPOW to determine excess production
# in general, given two series and a threshold (boolean) function, the fn will split the the x and y values
# based on the function and then do ols on each side, returning a new series with the residual
# of each y-value from the fitted line

def lr_sd(lr, x, y):
    return (y - lr.predict(x)).pow(2).sum() / y.size

class SDLinReg:
    def __init__(self):
        return None
    
    def fit(self, d, x, y, f=None):
        self.x = x
        self.y = y
        data = d[[x, y]].copy()
        if f is None:
            d1 = data
            lr1 = LinearRegression()
            lr1.fit(d1[x].values.reshape(-1, 1), d1[y])
            sd1 = lr_sd(lr1, d1[[x]], d1[y])
            self.p = lambda r : (r[1] - lr1.predict([[r[0]]])) / sd1
        else:
            d1, d2 = data[f(data[x], data[y])], data[~ f(data[x], data[y])]
            lr1, lr2 = LinearRegression(), LinearRegression()
            lr1.fit(d1[x].values.reshape(-1, 1), d1[y])
            lr2.fit(d2[x].values.reshape(-1, 1), d2[y])
            sd1, sd2 = lr_sd(lr1, d1[[x]], d1[y]), lr_sd(lr2, d2[[x]], d2[y])
            self.p = lambda r : ((r[1] - lr1.predict([[r[0]]])) / sd1) if f(r[0], r[1]) else ((r[1] - lr2.predict([[r[0]]])) / sd2)
        return

    def predict(self, d, x=None, y=None, debug=False):
        if x is None:
            x = self.x
        if y is None:
            y = self.y
        data = d[[x, y]]
        if debug:
            i = 0
            for row in data.itertuples(index=False):
                print("row:", row)
                print("p(row):", self.p(row))
                print("float:", float(self.p(row)))
                i += 1
                if i > 20:
                    break
        return pd.Series([float(self.p(row)) for row in data.itertuples(index=False)], index=d.index)

In [10]:
# Function of preprocessing the data
def preprocessing(df):
    # FIND COLUMNS WITH MISSING VALUE
    missing_values_count = df.isnull().sum()/len(df)*100
    columns_with_missing_values = missing_values_count[missing_values_count > 0].index.tolist()
    columns_with_missing_values = list(set(columns_with_missing_values))

    # INPUT MISSING VALUE WITH KNNIMPUTER
    knn_imputer = KNNImputer(n_neighbors=5)
    df[columns_with_missing_values] = knn_imputer.fit_transform(df[columns_with_missing_values])

    #Drop the columns that are not useful
    x = df.drop(['FR_DE_EXCHANGE', 'FR_NET_EXPORT', 'DE_NET_EXPORT'], axis=1).fillna(df.mean(numeric_only=True))


    #Factorize the COUNTRY column
    x['COUNTRY'] = x['COUNTRY'].factorize()[0]

    # Create the new columns
    x['DE_NON_RENEWABLE'] = x['DE_GAS'] + x['DE_COAL'] + x['DE_LIGNITE'] + x['DE_NUCLEAR']
    x['DE_RENEWABLE'] = x['DE_HYDRO'] + x['DE_SOLAR'] + x['DE_WINDPOW']
    x['DE_EXCESS_ENERGY'] = x['DE_NON_RENEWABLE'] + x['DE_RENEWABLE'] - x['DE_CONSUMPTION']

    # Create the new columns
    x['FR_NON_RENEWABLE'] = x['FR_GAS'] + x['FR_COAL'] + x['FR_NUCLEAR']
    x['FR_RENEWABLE'] = x['FR_HYDRO'] + x['FR_SOLAR'] + x['FR_WINDPOW']
    x['FR_EXCESS_ENERGY'] = x['FR_NON_RENEWABLE'] + x['FR_RENEWABLE'] - x['FR_CONSUMPTION']

    # Relation between the wind columns and the wind power column
    x['DE_WIND_SQCB'] = (x['DE_WIND'] - x['DE_WIND'].min()).pow(2.0/3.0)
    x['FR_WIND_SQCB'] = (x['FR_WIND'] - x['FR_WIND'].min()).pow(2.0/3.0)
    x['DE_WIND_CBRT'] = (x['DE_WIND'] - x['DE_WIND'].min()).pow(1.0/3.0)
    x['FR_WIND_CBRT'] = (x['FR_WIND'] - x['FR_WIND'].min()).pow(1.0/3.0)

    ## ADD DATE COL
    start_date = pd.to_datetime('2020-01-01')
    x['DATE'] = x['DAY_ID'].apply(lambda x: start_date + pd.Timedelta(days=x-1))
    
    # Threshold values for cut-in speeds of German and French wind turbines, respectively.
    de_wind_switch = 1.5
    fr_wind_switch = 1.5

    # some post-split work to determine excess energy production
    # measuring excess wind energy production, determined by forecasted production based on wind levels
    wind_excess_lr = SDLinReg()
    wind_excess_lr.fit(x, 'DE_WIND_SQCB', 'DE_WINDPOW', lambda x, y : x > de_wind_switch)
    x['DE_WIND_EXCESS'] = wind_excess_lr.predict(x)

    wind_excess_lr.fit(x, 'FR_WIND_SQCB', 'FR_WINDPOW', lambda x, y : x > fr_wind_switch)
    x['FR_WIND_EXCESS'] = wind_excess_lr.predict(x)

   # french residual load is highly correlated with its consumption—seems to turn on/off non-renewable plants
   # in response to projected consumption, hence we detect overproduction by seeing if consumption is lower than the
   # prediction
    fr_consumption_lr = SDLinReg()
    fr_consumption_lr.fit(x, 'FR_RESIDUAL_LOAD', 'FR_CONSUMPTION')
    x['FR_OVERCON'] = fr_consumption_lr.predict(x)

   #Normalize the data
    norm_cols = ['GAS_RET', 'COAL_RET', 'CARBON_RET']
    x[norm_cols] = (x[norm_cols] - x[norm_cols].mean()) / x[norm_cols].std()

   #Create a new feature that clusters the DE_NUCLEAR column
    km = KMeans(n_clusters=6)
    km.fit(np.array(x['DE_NUCLEAR']).reshape(-1, 1))
    x['DE_NUCLEAR_CLUSTER'] = km.labels_

    # One hot encode the DE_NUCLEAR_CLUSTER column
    nuclear_onehot = onehot()
    nuclear_onehot.fit(np.array(x['DE_NUCLEAR_CLUSTER']).reshape(-1, 1))
    nuclear_onehot_cols = ['DE_NUCLEAR_0', 'DE_NUCLEAR_1', 'DE_NUCLEAR_2', 'DE_NUCLEAR_3', 'DE_NUCLEAR_4', 'DE_NUCLEAR_5']
    x[nuclear_onehot_cols] = pd.DataFrame(nuclear_onehot.transform(np.array(x['DE_NUCLEAR_CLUSTER']).reshape(-1, 1)).toarray(), index=x.index)

    return x 
    

In [11]:
x_train = preprocessing(df)
x_test = preprocessing(true_test)

/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_64275/502714257.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return pd.Series([float(self.p(row)) for row in data.itertuples(index=False)], index=d.index)
/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_64275/502714257.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return pd.Series([float(self.p(row)) for row in data.itertuples(index=False)], index=d.index)
/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_64275/502714257.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you

In [12]:
x_train

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,DE_NET_IMPORT,FR_NET_IMPORT,DE_GAS,FR_GAS,DE_COAL,FR_COAL,DE_HYDRO,FR_HYDRO,DE_NUCLEAR,FR_NUCLEAR,DE_SOLAR,FR_SOLAR,DE_WINDPOW,FR_WINDPOW,DE_LIGNITE,DE_RESIDUAL_LOAD,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,DE_NON_RENEWABLE,DE_RENEWABLE,DE_EXCESS_ENERGY,FR_NON_RENEWABLE,FR_RENEWABLE,FR_EXCESS_ENERGY,DE_WIND_SQCB,FR_WIND_SQCB,DE_WIND_CBRT,FR_WIND_CBRT,DATE,DE_WIND_EXCESS,FR_WIND_EXCESS,FR_OVERCON,DE_NUCLEAR_CLUSTER,DE_NUCLEAR_0,DE_NUCLEAR_1,DE_NUCLEAR_2,DE_NUCLEAR_3,DE_NUCLEAR_4,DE_NUCLEAR_5
0,1054,206,0,0.210099,-0.427458,-0.606523,-0.074847,-0.692860,0.441238,-0.213766,0.740627,0.288782,2.209047,0.207838,0.709614,-0.190463,0.101766,1.248911,-0.573370,-0.269460,0.870344,0.626666,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.255896,0.060770,-0.075508,2.761824,1.737443,4.289168,-0.115448,1.187289,1.499299,1.058872,1.374902,1.029015,1.172562,2020-07-24,-0.424541,-1.081876,-2.063908,4,0.0,0.0,0.0,0.0,1.0,0.0
1,2049,501,0,-0.022399,-1.003452,-0.022063,0.573520,1.130838,0.174773,0.426940,-0.170392,-0.762153,0.187964,-0.807112,-1.882740,-2.185961,1.987428,3.237380,-0.035514,-0.107350,-0.194308,-0.395469,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.653341,-0.014132,-0.519628,-2.072667,2.139877,0.089610,-2.521174,2.322919,0.805198,2.262316,1.950336,1.504100,1.396544,2021-05-15,-0.802641,-0.491355,0.609125,3,0.0,0.0,0.0,1.0,0.0,0.0
2,1924,687,0,1.395035,1.978665,1.021305,0.622021,1.682587,2.351913,2.122241,1.572267,0.777053,-0.108578,0.779142,-1.897109,0.735137,-1.115583,-0.371039,-0.298755,-0.141239,0.428272,1.336625,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.435291,0.659294,0.113270,2.455344,-1.522916,-0.462607,3.634432,0.266865,1.922631,1.748246,1.784051,1.322213,1.335684,2021-11-17,-0.289006,-0.357727,-1.384961,3,0.0,0.0,0.0,1.0,0.0,0.0
3,297,720,1,-0.983324,-0.849198,-0.839586,0.270870,-0.563230,0.487818,0.194659,-1.473817,-0.786025,-0.368417,1.320483,-0.205547,-1.589554,1.751523,0.562944,-0.010090,0.366885,-2.330557,-1.191889,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.777510,-0.346174,0.904255,-3.522102,1.373016,-1.165763,-2.180920,2.250313,0.918591,1.240122,1.401447,1.113608,1.183827,2021-12-20,-0.173801,-0.380780,-0.250896,1,0.0,1.0,0.0,0.0,0.0,0.0
4,1101,818,0,0.143807,-0.617038,-0.924990,-0.036917,-0.990324,0.238693,-0.240862,1.003734,-0.274975,-0.230179,-0.795983,-0.005581,0.176935,0.693543,0.723587,-0.774941,-0.564498,0.691040,0.571613,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.170330,1.416915,2.306400,1.927887,-0.311578,1.472502,-0.338902,-0.636893,-0.358758,0.856207,0.920922,0.925315,0.959647,2022-03-28,-0.026802,-0.213282,-3.943384,1,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,459,809,1,1.529204,1.106682,-1.855327,0.218658,-1.450426,1.810665,1.388269,0.359723,-0.294001,-0.470809,1.708814,0.207266,0.712492,-1.325306,-0.855249,-0.057214,2.968535,0.790548,1.547782,0.509514,0.352199,0.393179,-0.540096,0.017301,-0.534811,-0.300242,0.745930,0.732981,1.128559,3.168202,-1.853329,-0.214331,1.806760,3.822101,4.522179,1.215644,1.540822,1.102562,1.241299,2022-03-19,-0.161827,2.000835,8.125044,1,0.0,1.0,0.0,0.0,0.0,0.0
1490,1674,887,0,1.618582,1.752840,0.611392,-0.449153,0.152146,1.972779,1.558300,0.561356,0.230746,-0.341147,2.957114,0.217922,0.926279,-1.222781,-0.923261,0.198857,0.789618,0.917482,1.358927,1.666252,1.036262,0.885839,-0.548776,-0.722460,0.129479,0.424804,0.796622,-0.142587,0.251082,3.669539,-1.365072,0.685885,2.715325,2.823470,3.785955,1.210390,1.112151,1.100177,1.054586,2022-06-05,0.262216,0.914461,-0.482733,1,0.0,1.0,0.0,0.0,0.0,0.0
1491,748,1083,1,0.856399,0.489199,-0.255778,1.5

In [14]:
# Save dataset
x_train.to_csv('../data/final_dataset/x_train.csv', index=False)
x_test.to_csv('../data/final_dataset/x_test.csv', index=False)

In [36]:
x_train

,ID,DE_CONSUMPTION,FR_CONSUMPTION,DE_NET_IMPORT,FR_NET_IMPORT,DE_GAS,FR_GAS,DE_COAL,FR_COAL,DE_HYDRO,FR_HYDRO,DE_NUCLEAR,FR_NUCLEAR,DE_SOLAR,FR_SOLAR,DE_WINDPOW,FR_WINDPOW,DE_LIGNITE,DE_RESIDUAL_LOAD,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND_SQCB,FR_WIND_SQCB,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,DE_WIND_EXCESS,FR_WIND_EXCESS,FR_OVERCON,DE_NUCLEAR_CLUSTER,DE_NUCLEAR_0,DE_NUCLEAR_1,DE_NUCLEAR_2,DE_NUCLEAR_3,DE_NUCLEAR_4,DE_NUCLEAR_5
0,1054,0.210099,-0.427458,-0.074847,-0.692860,0.441238,-0.213766,0.740627,0.288782,2.209047,0.207838,0.709614,-0.190463,0.101766,1.248911,-0.573370,-0.269460,0.870344,0.626666,-0.444661,-0.172680,-0.556356,1.058872,1.374902,-1.069070,-0.063404,0.255896,0.060770,-0.075508,-0.424541,-1.081876,-2.063908,3,0.0,0.0,0.0,1.0,0.0,0.0
1,2049,-0.022399,-1.003452,0.573520,1.130838,0.174773,0.426940,-0.170392,-0.762153,0.187964,-0.807112,-1.882740,-2.185961,1.987428,3.237380,-0.035514,-0.107350,-0.194308,-0.395469,-1.183194,-1.240300,-0.770457,2.262316,1.950336,0.437419,1.831241,-0.653341,-0.014132,-0.519628,-0.802641,-0.491355,0.609125,2,0.0,0.0,1.0,0.0,0.0,0.0
2,1924,1.395035,1.978665,0.622021,1.682587,2.351913,2.122241,1.572267,0.777053,-0.108578,0.779142,-1.897109,0.735137,-1.115583,-0.371039,-0.298755,-0.141239,0.428272,1.336625,1.947273,-0.480700,-0.313338,1.748246,1.784051,0.684884,0.114836,0.435291,0.659294,0.113270,-0.289006,-0.357727,-1.384961,2,0.0,0.0,1.0,0.0,0.0,0.0
3,297,-0.983324,-0.849198,0.270870,-0.563230,0.487818,0.194659,-1.473817,-0.786025,-0.368417,1.320483,-0.205547,-1.589554,1.751523,0.562944,-0.010090,0.366885,-2.330557,-1.191889,-0.976974,-1.114838,-0.507570,1.240122,1.401447,0.350938,-0.417514,0.777510,-0.346174,0.904255,-0.173801,-0.380780,-0.250896,1,0.0,1.0,0.0,0.0,0.0,0.0
4,1101,0.143807,-0.617038,-0.036917,-0.990324,0.238693,-0.240862,1.003734,-0.274975,-0.230179,-0.795983,-0.005581,0.176935,0.693543,0.723587,-0.774941,-0.564498,0.691040,0.571613,-0.526267,-0.541465,-0.424550,0.856207,0.920922,0.614338,0.729495,0.170330,1.416915,2.306400,-0.026802,-0.213282,-3.943384,1,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,459,1.529204,1.106682,0.218658,-1.450426,1.810665,1.388269,0.359723,-0.294001,-0.470809,1.708814,0.207266,0.712492,-1.325306,-0.855249,-0.057214,2.968535,0.790548,1.547782,0.509514,0.352199,0.393179,1.215644,1.540822,-0.534811,-0.300242,0.745930,0.732981,1.128559,-0.161827,2.000835,8.125044,1,0.0,1.0,0.0,0.0,0.0,0.0
1490,1674,1.618582,1.752840,-0.449153,0.152146,1.972779,1.558300,0.561356,0.230746,-0.341147,2.957114,0.217922,0.926279,-1.222781,-0.923261,0.198857,0.789618,0.917482,1.358927,1.666252,1.036262,0.885839,1.210390,1.112151,0.129479,0.424804,0.796622,-0.142587,0.251082,0.262216,0.914461,-0.482733,1,0.0,1.0,0.0,0.0,0.0,0.0
1491,748,0.856399,0.489199,1.531544,0.829568,2.108764,1.866399,1.072553,-0.180117,0.037892,0.277630,-1.833866,-0.964588,-0.626731,0.196789,-0.981718,0.303324,0.211422,1.493870,0.358120,0.207905,0.404763,1.182463,1.981527,0.256338,0.402316,-1.066732,-0.289750,-0.012159,-1.508300,-0.239596,0.048332,2,0.0,0.0,1.0,0.0,0.0,0.0
1492,1454,0.560689,-0.343777,0.304856,-1.210230,-0.003973,0.869742,-0.436935,-0.772801,-0.288150,1.447245,0.107706,-0.196405,0.619473,0.918251,0.378080,-0.973969,-0.828580,-0.137667,-0.184862,-0.682815,-0.390304,0.937914,0.536885,1.215528,1.338708,0.824114,-5.275969,-0.841346,1.509848,0.327306,-5.058292,1,0.0,1.0,0.0,0.0,0.0,0.0
